In [13]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import torch
from transformers import pipeline
from transformers import BertTokenizer, BertForSequenceClassification

plt.style.use('ggplot')
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')


In [14]:
# Juntando as planilhas com classificação manual.

# Planilhas com classificação manual
df_dani = pd.read_excel('../data/manual_eval_tweets.xlsx', sheet_name='Daniela Nomura')
df_dagna = pd.read_excel('../data/manual_eval_tweets.xlsx', sheet_name='Dagna')
df_jose = pd.read_excel('../data/manual_eval_tweets.xlsx', sheet_name='José Valdeir')
df_bruno = pd.read_excel('../data/manual_eval_tweets.xlsx', sheet_name='Bruno')
df_thiago = pd.read_excel('../data/manual_eval_tweets.xlsx', sheet_name='Thiago')

# Agrupando dataframes com classificação manual para ajuste fino
df_agrupado = pd.concat([df_dani, df_dagna, df_jose, df_bruno, df_thiago])

# Salvando arquivo consolidado
output_path = "../data/processed/"
output_file = f"manual_class-tweets.csv"

df_agrupado.to_csv(f"{output_path}{output_file}", sep=";", encoding="utf-8", index=False)
df_ajuste = df_agrupado[['content', 'classe_manual']]

## Utilizando dados classificados manualmente

In [15]:
torch.cuda.current_device()

0

In [16]:
classifier = pipeline(
    "sentiment-analysis",
    model='ruanchaves/bert-base-portuguese-cased-hatebr',
    device=torch.cuda.current_device(),
)

In [17]:
%%time
results = df_ajuste['content'].apply(lambda x: classifier(x))

CPU times: user 12 s, sys: 1.59 s, total: 13.5 s
Wall time: 13.6 s


In [18]:
# Separando rotulo e score
labels = [result[0]['label'] for result in results]
scores = [result[0]['score'] for result in results]

In [19]:
# Criando colunas com rótulos e score no dataframe de ajuste
df_ajuste['classif_sem_ajuste'] = labels
df_ajuste['score_sem_ajuste'] = scores

In [20]:
df_ajuste.iloc[:, [0,1,2]]

,content,classe_manual,classif_sem_ajuste
0,Não acredite nesses fantasmas que querem distr...,False,True
1,"Pelo jeito, a carruagem virou abóbora antes da...",False,True
2,"@rosana Gente, só eu não vi? Desculpa a vergon...",False,True
3,@victorpvianna Crime,False,False
4,O Alexandre tá passando pano?,False,True
...,...,...,...
396,@carteiroreaca 👏👏👏👏🇧🇷🇧🇷,False,False
397,@carteiroreaca E se fosse vc no lugar do Dougl...,False,False
398,"@carteiroreaca @carteiroreaca ,como eu te prom...",False,False
399,"@carteiroreaca É sensato da sua parte, deputado.",False,False


In [21]:
# criando coluna adicional para não ter perigo de errar e ter que fazer a classificação novamente
df_ajuste['classe_manual_2'] = df_ajuste['classe_manual']

In [22]:
# Respostas com classificação divergente
df_ajuste['classe_manual'].value_counts()

False         1035
FALSO          635
True           170
VERDADEIRO     160
Name: classe_manual, dtype: int64

In [23]:
# Padronizando respostas
df_ajuste['classe_manual_2'] = df_ajuste['classe_manual_2'].replace('FALSO', False)
df_ajuste['classe_manual_2'] = df_ajuste['classe_manual_2'].replace('VERDADEIRO', True)

In [24]:
# Verificando resultados da classificação manual
df_ajuste['classe_manual_2'].value_counts()

False    1670
True      330
Name: classe_manual_2, dtype: int64

In [25]:
# Verificando resultados da classificação do modelo pré-treinado antes do ajuste fino
df_ajuste['classif_sem_ajuste'].value_counts()

False    1463
True      537
Name: classif_sem_ajuste, dtype: int64

In [26]:
# Verificando métricas do modelo pré-treinado antes do ajuste fino
acuracia_pre_ajuste = accuracy_score(df_ajuste['classe_manual_2'], df_ajuste['classif_sem_ajuste'])
precision_pre_ajuste = precision_score(df_ajuste['classe_manual_2'], df_ajuste['classif_sem_ajuste'])
recall_pre_ajuste = recall_score(df_ajuste['classe_manual_2'], df_ajuste['classif_sem_ajuste'])
f1_pre_ajuste = f1_score(df_ajuste['classe_manual_2'], df_ajuste['classif_sem_ajuste'])

print("="*30)
print("Classificação Pré Ajuste Fino")
print("-"*30)
print("Acurácia:", acuracia_pre_ajuste.round(3))
print("Precision:", precision_pre_ajuste.round(3))
print("Recall:", recall_pre_ajuste.round(3))
print("F1 Score:", f1_pre_ajuste.round(3))
print("="*30)

Classificação Pré Ajuste Fino
------------------------------
Acurácia: 0.824
Precision: 0.48
Recall: 0.782
F1 Score: 0.595


# Ajuste fino do modelo escolhido

In [28]:
# Verificando dataframe que será usado para ajuste
df_ajuste

,content,classe_manual,classif_sem_ajuste,score_sem_ajuste,classe_manual_2
0,Não acredite nesses fantasmas que querem distr...,False,True,0.999933,False
1,"Pelo jeito, a carruagem virou abóbora antes da...",False,True,0.999743,False
2,"@rosana Gente, só eu não vi? Desculpa a vergon...",False,True,0.999871,False
3,@victorpvianna Crime,False,False,0.956910,False
4,O Alexandre tá passando pano?,False,True,0.994142,False
...,...,...,...,...,...
396,@carteiroreaca 👏👏👏👏🇧🇷🇧🇷,False,False,0.999860,False
397,@carteiroreaca E se fosse vc no lugar do Dougl...,False,False,0.999962,False
398,"@carteiroreaca @carteiroreaca ,como eu te prom...",False,False,0.999977,False
399,"@carteiroreaca É sensato da sua parte, deputado.",False,False,0.999978,False


In [29]:
# Convertendo classificação em binário para o formato adequado para ajuste fino do modelo
df_ajuste['classe_manual_bin'] = df_ajuste['classe_manual_2'].apply(lambda x: 0 if x == False else 1)
df_ajuste['classif_sem_ajuste_bin'] = df_ajuste['classif_sem_ajuste'].apply(lambda x: 0 if x == False else 1)

In [30]:
# Verificando a conversão
df_ajuste[['classe_manual_bin', 'classe_manual_2']].value_counts()

classe_manual_bin  classe_manual_2
0                  False              1670
1                  True                330
dtype: int64

In [31]:
# Verificando a conversão
df_ajuste[['classif_sem_ajuste', 'classif_sem_ajuste_bin']].value_counts()

classif_sem_ajuste  classif_sem_ajuste_bin
False               0                         1463
True                1                          537
dtype: int64

In [32]:
# Fazendo divisão entre treino e teste
X = df_ajuste['content']
Y = df_ajuste['classe_manual_bin']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size = 0.3, stratify = Y, random_state=42
)

In [33]:
# Fazendo divisão entre treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size = 0.3, stratify = y_train, random_state=42
)

In [34]:
# Verificando dimensões
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((980,), (420,), (600,), (980,), (420,), (600,))

In [35]:
# Carregando modelo BERT pré-treinado
model_name = "ruanchaves/bert-base-portuguese-cased-hatebr"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 rótulos: 'False', 'True'

In [36]:
# Convertendo dados para formato adequado e separando conjunto de treino e validação.

train_texts = X_train.to_list()  # Lista of tweet para treino
train_labels = y_train.to_list() # Lista of rótulos (0 ou 1) para treino

val_texts = X_val.to_list()    # Lista of tweet para validação
val_labels = y_val.to_list()   # Lista of rótulos (0 ou 1) para validação

# Tokenizar e converter dados para formato adequado para o BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convertendo dados para tensores do PyTorch
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings.input_ids),
    torch.tensor(train_encodings.attention_mask),
    torch.tensor(train_labels)
)

val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings.input_ids),
    torch.tensor(val_encodings.attention_mask),
    torch.tensor(val_labels)
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [37]:
# Definindo o DataLoader para batching
from torch.utils.data import DataLoader, RandomSampler

train_batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=train_batch_size)

val_batch_size = 32
val_dataloader = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=val_batch_size)

In [38]:
from transformers import AdamW

# Define otimizador e loss function. 
# OBS: Aqui também poderia ser testado outros parametros.
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()


# Foi estabelecido 3 épocas. Sem GPU levou por volta de 18min cada. Talvez com mais épocas, mais dados ou os dois conseguimos melhoras
num_epochs = 3   
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)

cuda


In [39]:
%%time
# Loop de ajuste fino
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validação
    model.eval()
    val_accuracy = 0
    val_loss = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            val_accuracy += torch.sum(preds == labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    avg_val_accuracy = val_accuracy / len(val_dataset)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train loss: {avg_train_loss:.4f}, Validation loss: {avg_val_loss:.4f}")
    print(f"Validation accuracy: {avg_val_accuracy:.4f}")

Epoch 1/3
Train loss: 0.4771, Validation loss: 0.2971
Validation accuracy: 0.9000
Epoch 2/3
Train loss: 0.2226, Validation loss: 0.2840
Validation accuracy: 0.8976
Epoch 3/3
Train loss: 0.1524, Validation loss: 0.2620
Validation accuracy: 0.9071
CPU times: user 39.4 s, sys: 18.8 s, total: 58.3 s
Wall time: 59.1 s


In [40]:
# Salvando o modelo. No meu caso foi salvo no ambiente virtual
output_dir = "./fine_tuned_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json')

## Classificando com o modelo refinado

In [ ]:
# Diretório onde modelo foi salvo
model_path = "./fine_tuned_model"

# Carregando tokenizer e modelo
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Escolhendo o dispositivo adequado (CPU or GPU) caso haja mais que um disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Criando pipeline customizado de analise de sentimento
# OBS: possível instanciar o modelo sem o tokenizer e device
classifier = pipeline(
    task="sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=torch.cuda.current_device()
)

In [ ]:
# Armazenando resultados da classificação
results = X_test.apply(lambda x: classifier(x))

In [ ]:
# Separando rotulo e score
labels_pos_ajuste = [result[0]['label'] for result in results]
scores_pos_ajuste = [result[0]['score'] for result in results]

In [ ]:
# Criando dataframe com tweets, classificação manual e do modelo pós ajuste fino
df_pos_ajuste = pd.DataFrame({'tweets': X_test, 'classe_manual_bin': y_test, 'classe_ajuste_fino': labels_pos_ajuste})

In [ ]:
# Convertendo resultados binários e booleanos
df_pos_ajuste['classe_manual'] = df_pos_ajuste['classe_manual_bin'].apply(lambda x: False if x == 0 else True)

In [ ]:
# Verificando dataframe
df_pos_ajuste

In [ ]:
# Verificando performance do modelo pré-treinado pós ajuste fino 
acuracia_pos_ajuste = accuracy_score(df_pos_ajuste['classe_manual'], df_pos_ajuste['classe_ajuste_fino'])
precision_pos_ajuste = precision_score(df_pos_ajuste['classe_manual'], df_pos_ajuste['classe_ajuste_fino'])
recall_pos_ajuste = recall_score(df_pos_ajuste['classe_manual'], df_pos_ajuste['classe_ajuste_fino'])
f1_pos_ajuste = f1_score(df_pos_ajuste['classe_manual'], df_pos_ajuste['classe_ajuste_fino'])

In [ ]:
# Criando dataframe comparativo entre modelos
modelos_dic = {
    'Modelo Pré Ajuste Fino': {
        "Acurácia": acuracia_pre_ajuste.round(3),
        "Precision": precision_pre_ajuste.round(3),
        "Recall": recall_pre_ajuste.round(3),
        "F1 Score": f1_pre_ajuste.round(3) 
    },
    'Modelo Pós Ajuste Fino': {
        "Acurácia": acuracia_pos_ajuste.round(3),
        "Precision": precision_pos_ajuste.round(3),
        "Recall": recall_pos_ajuste.round(3),
        "F1 Score": f1_pos_ajuste.round(3)
    }
}

df_modelos = pd.DataFrame(modelos_dic)
df_modelos

In [ ]:
# criando dataframe para gráfico
df_graficos_modelos = df_modelos.T.reset_index()
df_graficos_modelos['Acurácia'] = df_graficos_modelos['Acurácia']*100
df_graficos_modelos['Precision'] = df_graficos_modelos['Precision']*100
df_graficos_modelos['Recall'] = df_graficos_modelos['Recall']*100
df_graficos_modelos['F1 Score'] = df_graficos_modelos['F1 Score']*100
df_graficos_modelos.rename(columns = {'index': 'Modelos'}, inplace=True)

In [ ]:
plt.figure(figsize = (6,4))
ax = sns.barplot(df_graficos_modelos, x = 'Modelos', y = 'Acurácia', color = 'steelblue')
ax.set_title('Acurácia', fontsize=20, x=0.003, y=1.2, color="#696969")  # dimgrey
ax.spines['bottom'].set_color('#696969')  
ax.spines['left'].set_color('#696969')  
ax.tick_params(axis='x', colors='#696969')  
ax.tick_params(axis='y', colors='#696969')
ax.patches[0].set_facecolor('indianred')
ax.set_ylim(0, 100)
ax.set_ylabel("")
ax.set_xlabel("")

aumento_acc = 8.07

subtitle_text = rf"${{Aumento\ de}}$ " + \
               fr"${{\bf{aumento_acc}\%}}$ " + \
               r"${{em\ acurácia}}$"

ax.text(0.135, 1.12, subtitle_text, transform=ax.transAxes, fontsize=10, ha='center', color='#696969')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(decimals=0))

In [ ]:
plt.figure(figsize = (6,4))
ax = sns.barplot(df_graficos_modelos, x = 'Modelos', y = 'Precision', color = 'steelblue')
ax.set_title('Precision', fontsize=20, x=0.015, y=1.2, color="#696969")  # dimgrey
ax.spines['bottom'].set_color('#696969')  
ax.spines['left'].set_color('#696969')  
ax.tick_params(axis='x', colors='#696969')  
ax.tick_params(axis='y', colors='#696969')
ax.patches[0].set_facecolor('indianred')
ax.set_ylim(0, 100)
ax.set_ylabel("")
ax.set_xlabel("")

aumento_pc = 56.87

subtitle_text = rf"${{Aumento\ de}}$ " + \
               fr"${{\bf{aumento_pc}\%}}$ " + \
               r"${{em\ precision}}$"

ax.text(0.153, 1.12, subtitle_text, transform=ax.transAxes, fontsize=10, ha='center', color='#696969')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(decimals=0))

In [ ]:
plt.figure(figsize = (6,4))
ax = sns.barplot(df_graficos_modelos, x = 'Modelos', y = 'Recall', color = 'indianred')
ax.set_title('Recall', fontsize=20, x=-0.025, y=1.2, color="#696969")  # dimgrey
ax.spines['bottom'].set_color('#696969')  
ax.spines['left'].set_color('#696969')  
ax.tick_params(axis='x', colors='#696969')  
ax.tick_params(axis='y', colors='#696969')
ax.patches[0].set_facecolor('steelblue')
ax.set_ylim(0, 100)
ax.set_ylabel("")
ax.set_xlabel("")

aumento_pc = 17.37 

subtitle_text = rf"${{Diminuição\ de}}$ " + \
               fr"${{\bf{aumento_pc}\%}}$ " + \
               r"${{em\ recall}}$"

ax.text(0.142, 1.12, subtitle_text, transform=ax.transAxes, fontsize=10, ha='center', color='#696969')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(decimals=0))

In [ ]:
plt.figure(figsize = (6,4))
ax = sns.barplot(df_graficos_modelos, x = 'Modelos', y = 'F1 Score', color = 'steelblue')
ax.set_title('F1 Score', fontsize=20, x=0.013, y=1.2, color="#696969")  # dimgrey
ax.spines['bottom'].set_color('#696969')  
ax.spines['left'].set_color('#696969')  
ax.tick_params(axis='x', colors='#696969')  
ax.tick_params(axis='y', colors='#696969')
ax.patches[0].set_facecolor('indianred')
ax.set_ylim(0, 100)
ax.set_ylabel("")
ax.set_xlabel("")

aumento_acc = 16.97

subtitle_text = rf"${{Aumento\ de}}$ " + \
               fr"${{\bf{aumento_acc}\%}}$ " + \
               r"${{em\ F1 Score}}$"

ax.text(0.145, 1.12, subtitle_text, transform=ax.transAxes, fontsize=10, ha='center', color='#696969')

ax.yaxis.set_major_formatter(mticker.PercentFormatter(decimals=0))